In [ ]:
%run -i 'commons.ipynb'

In [ ]:
import socket
import pyspark
from pyspark.sql import SparkSession

localIpAddress = socket.gethostbyname(socket.gethostname())

conf = pyspark.SparkConf().setAll([
    ('spark.master', 'k8s://https://kubernetes.default.svc.cluster.local:443'),
    ("spark.driver.host", localIpAddress),
    ("spark.kubernetes.namespace", "spark"),
    ("spark.executor.instances", 5),
    ("spark.kubernetes.container.image", "akirillov/spark:spark-2.4.3-hadoop-2.9-k8s-horovod")])

spark = SparkSession.builder.appName("HorovodOnSpark").config(conf=conf).getOrCreate()

In [ ]:
from pyspark.sql import HiveContext, SQLContext, Row
from pyspark.sql.types import *

train = spark.read.csv("s3a://akirillov-dev/mnist_csv/mnist_train.csv", header=True, inferSchema=True)
test = spark.read.csv("s3a://akirillov-dev/mnist_csv/mnist_test.csv", header=True, inferSchema=True)

train.printSchema()

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["{}x{}".format(i,j) for i in range(1,29) for j in range (1,29)],
    outputCol="features")

labeledPoints = assembler.transform(train).select("label", "features")

In [ ]:
import pandas as pd
import numpy as np

pandaDf = labeledPoints.toPandas()

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(x_train.shape)
print(y_train.shape)